<a href="https://colab.research.google.com/github/CesarDora/Exploration_of_data_generated_by_passenger_trains/blob/main/Vers%C3%A3o_GITHUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Isto está formatado como código
```

Dissertação apresentada à Escola Politécnica da Universidade de São Paulo para obtenção do título de Mestre em Ciências

Título: Exploração de dados gerados em trens de passageiros: informações operacionais em estações

Autor: Cesar Augusto Doratiotto
Orientador: Prof. Dr. Claudio L. Marte


In [ ]:
import pandas as pd
import os
#configura google colab pra ler arquivos no drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def minha_funcao (df, file_name):
  try:
    #Cria coluna datetime juntando os valores de Data e Tempo
    df["datetime"] = df["Data"] + " " + df["Tempo"]

    #Converte formato de String para datetime
    df["datetime"] =  pd.to_datetime(df["datetime"])
    #df["datetime"]

    #ID do Registro, Data, Tempo e Tail_Car_Relay são desnecessários depois da criação de datetime e verificação do sentido do trem
    df = df.drop(columns=["ID do Registro", "Data", "Tempo", "Tail_Car_Relay"])
    #df.head

    #Renomeia colunas do dataframe para padronização
    column_mapping = {
      'Distância (km)': 'distance (km)',
      'Speed (km/h)': 'speed (km/h)',
      'All_Door_Closed': 'all_doors_closed',
      'TRAIN_WEIGHT (kg)': 'train_weight (kg)',
      'Head_Car_Relay': 'head_car_relay',
      'datetime': 'datetime'
    }

    #Renomeia as colunas do DataFrame com base no mapeamento
    df.columns = [column_mapping[col] if col in column_mapping else col for col in df.columns]

    # Garanti a ordem das colunas
    ordered_columns = ['distance (km)', 'datetime', 'speed (km/h)', 'head_car_relay', 'all_doors_closed', 'train_weight (kg)']
    df = df[ordered_columns]

    #Converte colunas de zeros e uns para True e False
    df['all_doors_closed'] = df['all_doors_closed'].astype('bool')
    df['head_car_relay'] = df['head_car_relay'].astype('bool')

    #Os registros com velocidades altas são desnecessários para nossa análise
    #Remove registros com velocidade > v_max (1km/h)
    v_max = 1
    df = df.drop(df[df["speed (km/h)"] > v_max].index)

    #Remove linhas duplicadas
    df = df.drop_duplicates()

    #Defino a função que diz o estado do trem, como variável categórica
    #Existem 3 possíveis estados:
    # Andando (A)
    # Parado com portas fechadas (PPF)
    # Parado com portas abertas (PPA)
    def evaluate_state(row):
      stopping_speed = 0.5 #ver extras no final do documento
      #Andando
      if row['speed (km/h)'] >= stopping_speed and row ['all_doors_closed'] == 1:
        return "A"
      #Parado com portas fechadas
      if row['speed (km/h)'] < stopping_speed and row['all_doors_closed'] == 1:
        return "PPF"
      #Parado com portas abertas
      if row['speed (km/h)'] < stopping_speed and row['all_doors_closed'] == 0:
        return "PPA"

        #Crio uma nova coluna com os valores baseados na função que defini acima
    df['train_state'] = df.apply(evaluate_state, axis=1)

    #Defino função que diz se houve transição de estado - se o valor da linha anterior e linha atual for diferente houve transição
    def evaluate_state_changes(row):
        index = df.index.get_loc(row.name)
        if index == 0:
          return False
        prev_row = df.iloc[index - 1]
        return (row['train_state'] != prev_row['train_state'])

        #Cria nova coluna com as mudanças de estado
    df['train_state_changed'] = df.apply(evaluate_state_changes, axis=1)

    #Em alguns casos temos velocidades próximas de 0 (todas menores que 0.5) em que o trem abre as portas
    #como as distâncias não mudam, sabemos que o trêm está parado e isto pode ser ruído do tacogerador ou irrelevantes deslocamentos
    df[(df['speed (km/h)'] < 1) & (df['speed (km/h)'] > 0) & (df['all_doors_closed'] == False)]

    df_stations1 = df[((df['train_state'] == "PPA") | (df['train_state'] == "A")) & (df['train_state_changed'] == True)]
    # Converta a coluna 'datetime' para o tipo datetime
    df_stations1['datetime'] = pd.to_datetime(df_stations1['datetime'])
    # Inicialize uma coluna para armazenar o tempo de abertura das portas
    df_stations1['Portas_abertas'] = 0
    # Inicialize uma variável para rastrear o tempo em segundos entre "PPA" e "A"
    tempo_abertura = 0
    # Percorra o DataFrame e calcule o tempo de abertura das portas
    for index, row in df_stations1.iterrows():
        if row['train_state'] == "PPA":
            # Encontre o próximo índice onde 'train_state' é 'A' após 'PPA'
            proximo_indice_a = df_stations1[(df_stations1['train_state'] == 'A') & (df_stations1.index > index)].index.min()
            if not pd.isna(proximo_indice_a):
                # Calcule o tempo de abertura das portas
                tempo_abertura = (df_stations1.at[proximo_indice_a, 'datetime'] - row['datetime']).total_seconds()
                df_stations1.at[index, 'Portas_abertas'] = tempo_abertura

    df_stations = df_stations1[(df['train_state'] == "PPA") & (df['train_state_changed'] == True)]

    #Defino função que diz se é inicio ou fim da linha
    def evaluate_start_or_end(row):
        index = df_stations.index.get_loc(row.name)
        if index == 0:
          return False
        prev_row = df_stations.iloc[index - 1]
        return (row['head_car_relay'] != prev_row['head_car_relay'])

    #Calcula distância percorrida desde a parada anterior
    df_stations['distance_from_previous (km)'] = df_stations['distance (km)'] - df_stations.shift(1)['distance (km)']

    #Como está em km, mas o registrador marca até 6 casas decimais, uso o arredondamento para na comparação com a referência criar comparação correta
    df_stations['distance_from_previous (km)'] = df_stations['distance_from_previous (km)'].round(3)

    # Filtra as linhas onde a distância seja superior ou igual a 0.8, por causa das voltas de comando
    df_stations = df_stations[(df_stations['distance_from_previous (km)'] >= 0.8)]

    #diz se é inicio ou fim da linha
    df_stations['start_or_end'] = df_stations.apply(evaluate_start_or_end, axis=1)
    df_stations

    #colocar localização do arquivo no Google drive, começando com /content/gdrive
    dir = "/content/gdrive/My Drive/teste dissertacao/Distânicas estações_elaborada DataSet_usp.xlsx"
    #lê arquivo e cria dataframe
    df_distances = pd.read_excel(dir)
    #utilizo somente as colunas que me importam
    df_distances = df_distances[['Estação Origem', 'Estação de destino', 'Distânica (km)']]
    #renomeia colunas do dataframe para padronização
    #nomes em Inglês, em snake_case e com unidades entre parênteses separadas por espaços
    df_distances = df_distances.set_axis(['origin_station', 'destination_station','distance (km)'], axis=1)
    #excluo linhas que não tem nome de estações. Na planilha são as linhas antes do total
    df_distances = df_distances.dropna()
    df_distances
    #manipula a lista de distâncias para gerar um dataframe do sentido oposto
    df_distances_opposite = df_distances[::-1]
    df_distances_opposite = df_distances_opposite.reset_index(drop=True)
    df_distances_opposite = df_distances_opposite.set_axis(['destination_station', 'origin_station', 'distance (km)'], axis=1)
    df_distances_opposite = df_distances_opposite[['origin_station', 'destination_station','distance (km)']]
    df_distances_opposite

  # Adicionar uma coluna para o nome das estações, se ainda não estiver presente
    df_stations['station_name'] = ""

    epsilon = 0.25  # variação de 250m para classificação
    epsilon2 = 0.15 # variação de 150m para validação
    first_station_dist = df_distances.at[0, 'distance (km)']
    first_station_dist_opposite = df_distances_opposite.at[0, 'distance (km)']
    start_found = False
    station_index = 0

    # Primeiro, mapeia as estações começando pelo primeiro "start_or_end=True"
    for i in df_stations.index:
        if df_stations.at[i, 'start_or_end']:
            station_index = 0
            station_dist = df_stations.at[i, 'distance_from_previous (km)']

            # Validação para a direção normal
            if first_station_dist - epsilon <= station_dist <= first_station_dist + epsilon:
                if station_index < len(df_distances):
                    df_stations.at[i, 'station_name'] = df_distances.at[station_index, 'destination_station']
                    print("Achei um início!")
                    start_found = True
                    opposite = False
            # Validação para a direção oposta
            elif first_station_dist_opposite - epsilon <= station_dist <= first_station_dist_opposite + epsilon:
                if station_index < len(df_distances_opposite):
                    df_stations.at[i, 'station_name'] = df_distances_opposite.at[station_index, 'destination_station']
                    print("Achei um início!")
                    start_found = True
                    opposite = True
        elif start_found and not opposite:
            station_index += 1
            if station_index < len(df_distances):  # Verificar se o índice é válido
                station_dist = df_stations.at[i, 'distance_from_previous (km)']
                ref_dist = df_distances.at[station_index, 'distance (km)']

                # Validação da distância
                if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                    df_stations.at[i, 'station_name'] = df_distances.at[station_index, 'destination_station']
                else:
                    df_stations.at[i, 'station_name'] = "Não classificada"
            else:
                df_stations.at[i, 'station_name'] = "Não classificada"
        elif start_found and opposite:
            station_index += 1
            if station_index < len(df_distances_opposite):  # Verificar se o índice é válido
                station_dist = df_stations.at[i, 'distance_from_previous (km)']
                ref_dist = df_distances_opposite.at[station_index, 'distance (km)']

                # Validação da distância
                if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                    df_stations.at[i, 'station_name'] = df_distances_opposite.at[station_index, 'destination_station']
                else:
                    df_stations.at[i, 'station_name'] = "Não classificada"
            else:
                df_stations.at[i, 'station_name'] = "Não classificada"

    # Validação final para linhas antes da primeira estação nomeada
    if not df_stations['station_name'].str.strip().any():  # Nenhuma estação foi mapeada
        print("Nenhuma estação foi mapeada. Verifique os dados de entrada ou ajuste os critérios.")
    else:
        # Identificar as linhas antes da primeira estação nomeada
        first_named_index = df_stations[df_stations['station_name'] != ""].index[0]

        # Calcular o número de linhas sem nome antes da primeira estação
        lines_to_fill = df_stations.loc[:first_named_index - 1]

        # Reverter a tabela de distâncias correspondente
        if 'opposite' in locals() and opposite:
            reversed_distances = df_distances_opposite[::-1]
        else:
            reversed_distances = df_distances[::-1]

        # Atribuir os nomes às linhas sem nome na ordem inversa
        for i, station_index in zip(lines_to_fill.index[::-1], range(len(lines_to_fill))):
            if station_index < len(reversed_distances):
                ref_dist = reversed_distances.iloc[station_index]['distance (km)']
                station_dist = df_stations.at[i, 'distance_from_previous (km)']

                # Validação da distância
                if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                    df_stations.at[i, 'station_name'] = reversed_distances.iloc[station_index]['destination_station']
                else:
                    df_stations.at[i, 'station_name'] = "Não classificada"
            else:
                df_stations.at[i, 'station_name'] = "Não classificada"

    # Preencher todos os valores vazios como "Não classificada"
    df_stations['station_name'] = df_stations['station_name'].replace("", "Não classificada")

    # Garantir que os índices do DataFrame sejam contínuos
    df_stations = df_stations.reset_index(drop=True)

    # Continuar classificação das estações "Não classificadas"
    for i in df_stations.index:
        if df_stations.at[i, 'station_name'] == "Não classificada":
            station_dist = df_stations.at[i, 'distance_from_previous (km)']
            station_index = 0
            start_found = False
            opposite = False

            # Tentar encontrar a estação na tabela referencial
            for j in range(len(df_distances)):
                ref_dist = df_distances.at[j, 'distance (km)']

                if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                    # Verificar as próximas duas estações sem ultrapassar os limites
                    if (
                        i + 1 in df_stations.index and
                        i + 2 in df_stations.index and
                        j + 1 < len(df_distances) and
                        j + 2 < len(df_distances) and
                        abs(df_distances.at[j + 1, 'distance (km)'] - df_stations.at[i + 1, 'distance_from_previous (km)']) <= epsilon2 and
                        abs(df_distances.at[j + 2, 'distance (km)'] - df_stations.at[i + 2, 'distance_from_previous (km)']) <= epsilon2
                    ):
                        # Estação encontrada e verificada
                        df_stations.at[i, 'station_name'] = df_distances.at[j, 'destination_station']
                        print(f"Classifiquei a estação na linha {i} como {df_stations.at[i, 'station_name']}.")
                        start_found = True
                        opposite = False
                        break

            # Verificar na tabela oposta, caso necessário
            if df_stations.at[i, 'station_name'] == "Não classificada":
                for j in range(len(df_distances_opposite)):
                    ref_dist = df_distances_opposite.at[j, 'distance (km)']

                    if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                        # Verificar as próximas duas estações sem ultrapassar os limites
                        if (
                            i + 1 in df_stations.index and
                            i + 2 in df_stations.index and
                            j + 1 < len(df_distances_opposite) and
                            j + 2 < len(df_distances_opposite) and
                            abs(df_distances_opposite.at[j + 1, 'distance (km)'] - df_stations.at[i + 1, 'distance_from_previous (km)']) <= epsilon2 and
                            abs(df_distances_opposite.at[j + 2, 'distance (km)'] - df_stations.at[i + 2, 'distance_from_previous (km)']) <= epsilon2
                        ):
                            # Estação encontrada e verificada
                            df_stations.at[i, 'station_name'] = df_distances_opposite.at[j, 'destination_station']
                            print(f"Classifiquei a estação na linha {i} como {df_stations.at[i, 'station_name']} (tabela oposta).")
                            start_found = True
                            opposite = True
                            break

            # Classificar as estações subsequentes
            if start_found:
                for k in range(i + 1, len(df_stations)):
                    if df_stations.at[k, 'station_name'] == "Não classificada":
                        station_index += 1

                        try:
                            if not opposite:
                                if station_index < len(df_distances):
                                    station_dist = df_stations.at[k, 'distance_from_previous (km)']
                                    ref_dist = df_distances.at[station_index, 'distance (km)']

                                    if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                                        candidate_station = df_distances.at[station_index, 'destination_station']
                                        prev_station = df_stations.at[k - 1, 'station_name']

                                        if prev_station in df_distances['origin_station'].values:
                                            valid_next_stations = df_distances.loc[
                                                df_distances['origin_station'] == prev_station, 'destination_station'
                                            ].values

                                            if candidate_station in valid_next_stations:
                                                df_stations.at[k, 'station_name'] = candidate_station
                                            else:
                                                print(f"A estação {candidate_station} não é válida após {prev_station}. Linha {k} classificada como 'Não classificada'.")
                                        else:
                                            print(f"A estação anterior {prev_station} não está na tabela de referências. Linha {k} classificada como 'Não classificada'.")
                                    else:
                                        break
                                else:
                                    break
                            else:
                                if station_index < len(df_distances_opposite):
                                    station_dist = df_stations.at[k, 'distance_from_previous (km)']
                                    ref_dist = df_distances_opposite.at[station_index, 'distance (km)']

                                    if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                                        candidate_station = df_distances_opposite.at[station_index, 'destination_station']
                                        prev_station = df_stations.at[k - 1, 'station_name']

                                        if prev_station in df_distances_opposite['origin_station'].values:
                                            valid_next_stations = df_distances_opposite.loc[
                                                df_distances_opposite['origin_station'] == prev_station, 'destination_station'
                                            ].values

                                            if candidate_station in valid_next_stations:
                                                df_stations.at[k, 'station_name'] = candidate_station
                                            else:
                                                print(f"A estação {candidate_station} não é válida após {prev_station}. Linha {k} classificada como 'Não classificada'.")
                                        else:
                                            print(f"A estação anterior {prev_station} não está na tabela de referências oposta. Linha {k} classificada como 'Não classificada'.")
                                    else:
                                        break
                                else:
                                    break
                        except KeyError as e:
                            print(f"Erro ao acessar os índices para a estação {k} na linha {i}: {e}. Verifique os tamanhos das tabelas.")
                            break
    # Classificação das estações restantes verificando as duas estações anteriores
    for i in df_stations.index:
        if df_stations.at[i, 'station_name'] == "Não classificada":
            station_dist = df_stations.at[i, 'distance_from_previous (km)']
            classified = False  # Flag para verificar se a estação foi classificada

            # Verificar as duas estações anteriores na tabela referencial normal
            for j in range(len(df_distances) - 1, -1, -1):  # Iterar de trás para frente
                ref_dist = df_distances.at[j, 'distance (km)']

                if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                    # Verificar as duas estações anteriores
                    if (
                        i - 1 in df_stations.index and
                        i - 2 in df_stations.index and
                        j - 1 >= 0 and
                        j - 2 >= 0 and
                        abs(df_distances.at[j - 1, 'distance (km)'] - df_stations.at[i - 1, 'distance_from_previous (km)']) <= epsilon2 and
                        abs(df_distances.at[j - 2, 'distance (km)'] - df_stations.at[i - 2, 'distance_from_previous (km)']) <= epsilon2
                    ):
                        df_stations.at[i, 'station_name'] = df_distances.at[j, 'destination_station']
                        print(f"Classifiquei a estação na linha {i} como {df_stations.at[i, 'station_name']} (baseando-se nas anteriores).")
                        classified = True
                        break

            # Verificar na tabela referencial oposta, caso ainda não tenha sido classificada
            if not classified:
                for j in range(len(df_distances_opposite) - 1, -1, -1):
                    ref_dist = df_distances_opposite.at[j, 'distance (km)']

                    if ref_dist - epsilon <= station_dist <= ref_dist + epsilon:
                        # Verificar as duas estações anteriores
                        if (
                            i - 1 in df_stations.index and
                            i - 2 in df_stations.index and
                            j - 1 >= 0 and
                            j - 2 >= 0 and
                            abs(df_distances_opposite.at[j - 1, 'distance (km)'] - df_stations.at[i - 1, 'distance_from_previous (km)']) <= epsilon2 and
                            abs(df_distances_opposite.at[j - 2, 'distance (km)'] - df_stations.at[i - 2, 'distance_from_previous (km)']) <= epsilon2
                        ):
                            df_stations.at[i, 'station_name'] = df_distances_opposite.at[j, 'destination_station']
                            print(f"Classifiquei a estação na linha {i} como {df_stations.at[i, 'station_name']} (baseando-se nas anteriores - tabela oposta).")
                            classified = True
                            break

            # Caso nenhuma correspondência seja encontrada, manter como "Não classificada"
            if not classified:
                print(f"A estação na linha {i} permaneceu como 'Não classificada'.")

    # Exibir o DataFrame atualizado
    print(df_stations)

    # Preencher todos os valores vazios como "Não classificada"
    df_stations['station_name'] = df_stations['station_name'].replace("", "Não classificada")

    # Exibir o DataFrame atualizado
    print(df_stations)


    from datetime import datetime
    import pytz
    local_timezone = pytz.timezone('America/Sao_Paulo')
    data_hora_atual = datetime.now(local_timezone)
    nome_arquivo = data_hora_atual.strftime("%Y-%m-%d-%H-%M-%S")
    nome_arquivo = os.path.splitext(file_name)[0] + "_" + nome_arquivo + ".xlsx"

    # Adiciona a coluna "file_name" antes de salvar
    df_stations["file_name"] = file_name

    df_stations.to_excel(nome_arquivo)
    df_stations.to_excel("/content/gdrive/My Drive/teste dissertacao/automatico/" + nome_arquivo, index=False)

    df_stations.to_excel("output.xlsx")

    return

  except Exception as e:
        # Cria o nome do arquivo de erro com "_error.txt"
        error_file_name = os.path.splitext(file_name)[0] + "_error.txt"

        # Define o caminho do diretório correto para salvar o log de erro
        error_log_path = f"/content/gdrive/My Drive/teste dissertacao/automatico/{error_file_name}"

        # Salva a mensagem de erro no arquivo
        with open(error_log_path, "w") as log_file:
            log_file.write(f"Erro ao processar o arquivo {file_name}:\n{e}")

        print(f"Erro ao processar o arquivo {file_name}: {e}")
        return None

dir = "/content/gdrive/My Drive/teste dissertacao/Base_automatica"
arquivos = os.listdir(dir)

# Função para consolidar todas as abas de um arquivo Excel
def consolidar_abas(caminho_arquivo):
    # Carrega todas as abas do arquivo em um dicionário
    todas_abas = pd.read_excel(caminho_arquivo, sheet_name=None)

    # Lista para armazenar os DataFrames consolidados
    dataframes = []

    for idx, (nome_aba, df_aba) in enumerate(todas_abas.items()):
        if idx == 0:
            # Na primeira aba, mantém o cabeçalho
            dataframes.append(df_aba)
        else:
            # Nas demais abas, remove a primeira linha antes de adicionar
            df_aba_sem_cabecalho = df_aba.iloc[1:].reset_index(drop=True)
            dataframes.append(df_aba_sem_cabecalho)

    # Concatena todas as abas em um único DataFrame
    return pd.concat(dataframes, ignore_index=True)

# Processa todos os arquivos Excel no diretório
for arquivo in arquivos:
    if arquivo.endswith('.xls') or arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(dir, arquivo)
        df_consolidado = consolidar_abas(caminho_arquivo)

        # Chama a função para processar o DataFrame consolidado
        minha_funcao(df_consolidado, arquivo)


<ipython-input-2-42cfc6338194>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["datetime"] =  pd.to_datetime(df["datetime"])
<ipython-input-2-42cfc6338194>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['datetime'] = pd.to_datetime(df_stations1['datetime'])
<ipython-input-2-42cfc6338194>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations

Achei um início!
Classifiquei a estação na linha 1 como Jaraguá.
Classifiquei a estação na linha 2 como Vila Aurora.
Classifiquei a estação na linha 3 como Perus .
A estação Guapituba não é válida após Perus . Linha 4 classificada como 'Não classificada'.
Classifiquei a estação na linha 4 como Caieiras.
Classifiquei a estação na linha 5 como Franco da Rocha.
Classifiquei a estação na linha 6 como Baltazar Fidélis (baseando-se nas anteriores).
Classifiquei a estação na linha 7 como Franscico Morato (baseando-se nas anteriores).
Classifiquei a estação na linha 8 como Botujuru (baseando-se nas anteriores).
A estação na linha 9 permaneceu como 'Não classificada'.
A estação na linha 10 permaneceu como 'Não classificada'.
    distance (km)            datetime  speed (km/h)  head_car_relay  \
0    1.028560e+06 2024-06-05 07:13:59           0.0           False   
1    1.028563e+06 2024-06-05 07:16:46           0.0           False   
2    1.028565e+06 2024-06-05 07:19:53           0.0          

<ipython-input-2-42cfc6338194>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["datetime"] =  pd.to_datetime(df["datetime"])
<ipython-input-2-42cfc6338194>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['train_state'] = df.apply(evaluate_state, axis=1)
<ipython-input-2-42cfc6338194>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['datetime'] = 

Achei um início!
Achei um início!
    distance (km)            datetime  speed (km/h)  head_car_relay  \
0    1.028629e+06 2024-06-05 09:07:11           0.0            True   
1    1.028633e+06 2024-06-05 09:11:13           0.0            True   
2    1.028635e+06 2024-06-05 09:15:55           0.0            True   
3    1.028640e+06 2024-06-05 09:21:18           0.0            True   
4    1.028645e+06 2024-06-05 09:25:35           0.0            True   
..            ...                 ...           ...             ...   
60   1.028826e+06 2024-06-05 13:41:13           0.0            True   
61   1.028830e+06 2024-06-05 13:45:21           0.0            True   
62   1.028832e+06 2024-06-05 13:48:39           0.0            True   
63   1.028837e+06 2024-06-05 13:54:03           0.0            True   
64   1.028842e+06 2024-06-05 13:58:22           0.0            True   

    all_doors_closed  train_weight (kg) train_state  train_state_changed  \
0              False             3773

<ipython-input-2-42cfc6338194>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["datetime"] =  pd.to_datetime(df["datetime"])
<ipython-input-2-42cfc6338194>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['datetime'] = pd.to_datetime(df_stations1['datetime'])
<ipython-input-2-42cfc6338194>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations

...

<ipython-input-2-42cfc6338194>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["datetime"] =  pd.to_datetime(df["datetime"])
<ipython-input-2-42cfc6338194>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['datetime'] = pd.to_datetime(df_stations1['datetime'])
<ipython-input-2-42cfc6338194>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations

Nenhuma estação foi mapeada. Verifique os dados de entrada ou ajuste os critérios.
Classifiquei a estação na linha 0 como Guapituba.
Classifiquei a estação na linha 1 como Mauá.
Classifiquei a estação na linha 2 como Capuava.
Classifiquei a estação na linha 3 como Santo André.
Classifiquei a estação na linha 4 como Prefeito Saladino.
Classifiquei a estação na linha 5 como Utinga.
Classifiquei a estação na linha 6 como São Caetano do Sul.
Classifiquei a estação na linha 7 como Tamanduateí.
Classifiquei a estação na linha 8 como Ipiranga.
Classifiquei a estação na linha 9 como Mooca.
Classifiquei a estação na linha 10 como Brás.
Classifiquei a estação na linha 11 como Luz.
Classifiquei a estação na linha 12 como Barra-Funda.
Classifiquei a estação na linha 13 como Água Branca.
Classifiquei a estação na linha 14 como Lapa.
Classifiquei a estação na linha 15 como Piqueri.
Classifiquei a estação na linha 16 como Pirituba.
Classifiquei a estação na linha 17 como Vila Clarice.
Classifiquei a 

<ipython-input-2-42cfc6338194>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["datetime"] =  pd.to_datetime(df["datetime"])


Achei um início!
Classifiquei a estação na linha 0 como Caieiras.
Classifiquei a estação na linha 1 como Franco da Rocha.
Classifiquei a estação na linha 2 como Baltazar Fidélis.
Classifiquei a estação na linha 3 como Franscico Morato.
Classifiquei a estação na linha 4 como Botujuru.
Classifiquei a estação na linha 5 como Campo Limpo Paulista.
Classifiquei a estação na linha 6 como Várzea Paulista (baseando-se nas anteriores).
Classifiquei a estação na linha 7 como Jundiaí (baseando-se nas anteriores).
A estação na linha 38 permaneceu como 'Não classificada'.
A estação na linha 39 permaneceu como 'Não classificada'.
    distance (km)            datetime  speed (km/h)  head_car_relay  \
0    1.008495e+06 2024-06-05 19:11:30        0.0000           False   
1    1.008500e+06 2024-06-05 19:18:51        0.0000           False   
2    1.008502e+06 2024-06-05 19:28:49        0.4935           False   
3    1.008506e+06 2024-06-05 19:32:41        0.0000           False   
4    1.008513e+06 202

<ipython-input-2-42cfc6338194>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['datetime'] = pd.to_datetime(df_stations1['datetime'])
<ipython-input-2-42cfc6338194>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations1['Portas_abertas'] = 0
<ipython-input-2-42cfc6338194>:122: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_stations = df_stations1[(df['train_state'] == "PPA") & (df['train_state_changed'] == True)]
<ipython-input-2-42cfc6338194>:147: 